<B>1. Start by creating a new Notebook for this assignment.</br>

In [1]:
#*****     Segmenting and Clustering Neighborhoods in Toronto
#*****     Import libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
#install BeautifulSoup and LXML
!conda install -c anaconda beautifulsoup4 --yes
!conda install -c anaconda lxml --yes
!conda install -c conda-forge geocoder --yes
import bs4 as bs
import urllib.request
print("Done loading packages.")

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |           py36_0         165 KB  anaconda
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    soupsieve-2.0.1            |             py_0          33 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.3 MB

The following NEW packages will be INSTALLED:

  beautifulsoup4     anaconda/linux-64::beautifulsoup4-4.9.0-py36_0
  soupsieve          anaconda/noarch::soupsieve-2.0.1-py_0

The following packag

<B>2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [33]:
# setting the url and calling the page, then viewing the results of the first table.
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = pd.read_html(url, header=0)

<B>3. To create the dataframe:<BR>

In [34]:
# Copy the dataframe to pcs_df 
pcs_df = results[0]

# Rename the column Postal Code
pcs_df.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
pcs_df.head(4)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village


<B>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [35]:
# Exclude all Boroughs that equal 'Not Assigned'
pcs_df = pcs_df.loc[pcs_df['Borough'] != 'Not assigned']

<B>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [105]:
# I am using the GroupBy function to combine neighborhoods and then reset the index
pcs_df = pcs_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)).reset_index()

# List the first ten entries of the dataframe
pcs_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<B>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [37]:
# For every entry in the Neighborhood column that is blank, replace the entry with the name of the corresponding Borough
pcs_df.loc[pcs_df['Neighborhood'] == '', 'Neighborhood'] = pcs_df['Borough']  

<B>Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.</B><BR>
Done!

<B>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [38]:
print("The Toronto Postal Code dataframe has {} rows.".format(pcs_df.shape[0]))

The Toronto Postal Code dataframe has 103 rows.


<B>2. Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks)

In [113]:
#The geocoder code did not work, the response never came so I resorted to using the CSV

In [115]:
# Pull down the zipcode information
url = 'https://cocl.us/Geospatial_data'
pcs_ref = pd.read_csv(url)

In [116]:
#Sort the dataframe by Postal Code so the two match
pcs_ref = pcs_ref.sort_values('Postal Code')

In [117]:
#Sort the dataframe by PostalCode so the two match
pcs_df = pcs_df.sort_values('PostalCode')

In [118]:
#Concatenate the two dataframes vertically
pcs_df = pd.concat([pcs_df, zipcodes], axis=1, sort=False)

#Drop the second column labeled 'Postal Code'
pcs_df.drop(['Postal Code'], axis=1)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


43.7590508 -79.3102297


address = 'Malvern, Toronto, Ontario'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)